In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader
cudnn.benchmark = True
plt.ion()   # interactive mode
from utils.ImagesDataset import ImagesDataset
from tqdm import tqdm

# Dataset

In [2]:

dataset = ImagesDataset()
dataset.__len__()

215057

### Split

In [3]:
l=dataset.__len__()
val_split=0.3
train_set, val_set = torch.utils.data.random_split(dataset, [l-int(val_split*l), int(val_split*l)],generator=torch.Generator().manual_seed(42))
device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu")

### dataloader

In [4]:
bs=128
dataset_sizes = {'train':l-int(val_split*l),'val': int(val_split*l)}
train_loader = DataLoader(dataset=train_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
val_loader = DataLoader(dataset=val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
dataloaders={'train':train_loader,'val':val_loader}

# Training

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
   
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            with tqdm(dataloaders[phase], unit="batch") as tepoch:
                batches=0
                for inputs, labels in tepoch:
                    tepoch.set_description(f"Epoch {epoch}")
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    
                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        #print('out',outputs)
                        #print('labels',labels)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    batches+=1                      
                    tepoch.set_postfix(loss=running_loss/(batches*bs), accuracy=100. * running_corrects.item()/(batches*bs))

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### setup

In [6]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 15)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 2 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1)

In [7]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=4)

Epoch 0/3
----------


Epoch 0: 100%|███████████████████████████████████| 1177/1177 [07:10<00:00,  2.73batch/s, accuracy=81.9, loss=0.551]


train Loss: 0.5515 Acc: 0.8198


Epoch 0: 100%|█████████████████████████████████████| 505/505 [01:12<00:00,  6.95batch/s, accuracy=88.5, loss=0.343]


val Loss: 0.3435 Acc: 0.8867

Epoch 1/3
----------


Epoch 1:  51%|██████████████████▎                 | 598/1177 [03:39<03:32,  2.72batch/s, accuracy=90.9, loss=0.278]


KeyboardInterrupt: 

In [ ]:
torch.save(model_ft.state_dict(), './models/resnet18.pt')

### Resnet101

In [ ]:
model_ft = models.resnet101(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 15)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 2 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=12)
torch.save(model_ft.state_dict(), './models/resnet101.pt')

# Evaluation

In [14]:
#resnet model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 15)
model_ft = model_ft.to(device)


#reloading saved weights
model_ft.load_state_dict(torch.load('./models/resnet18.pt'))
model_ft.eval()



#evaluation
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    with tqdm(dataloaders['val'], unit="batch") as tepoch:
        for images, labels in tepoch:
            tepoch.set_description('progress lolol')
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_ft(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Validation accuracy : {acc} %')

progress lolol: 100%|██████████████████████████████████████████████████████| 505/505 [01:06<00:00,  7.60batch/s]

Validation accuracy : 93.13514267557387 %
